In [9]:
import csv
import sys
import time
import numpy as np
from libsvm.svmutil import *
from util import *

In [3]:
kernel = {
    'linear': 0, 
    'polynomial': 1, 
    'RBF': 2,
}

In [10]:
X_train = openCSV('data/X_train.csv').astype(np.float64)
Y_train = list(openCSV('data/Y_train.csv').astype(np.int32).ravel())
X_test = openCSV('data/X_test.csv').astype(np.float64)
Y_test = list(openCSV('data/Y_test.csv').astype(np.int32).ravel())

# Part 1

In [14]:
# linear kernel
m =  svm_train(Y_train, X_train, f'-t 0 -d 2 -q')
res = svm_predict(Y_test, X_test, m)
# polynomial kernel
m =  svm_train(Y_train, X_train, f'-t 1 -d 2 -q')
res = svm_predict(Y_test, X_test, m)
# RBF kernel
m =  svm_train(Y_train, X_train, f'-t 2 -d 2 -q')
res = svm_predict(Y_test, X_test, m)


Accuracy = 95.08% (2377/2500) (classification)
Accuracy = 88.24% (2206/2500) (classification)
Accuracy = 95.32% (2383/2500) (classification)


# Part 2 GridSearch

In [6]:
def compareAccuracy(Y, X, opt, optimal_cv_acc, optimal_opt):
    print(opt)
    cv_acc = svm_train(Y, X, opt)
    if cv_acc > optimal_cv_acc:
        return cv_acc, opt
    else:
        return optimal_cv_acc, optimal_opt


In [7]:
def gridSearch(X, Y):
    costs = [0.001, 0.01, 0.1, 1, 10]
    gammas = [0.001, 0.01, 0.1, 1]
    cnt = 0
    optimal_cv_acc = 0
    optimal_opt = ''
    for k in kernel:
        for c in costs:
            if k == 'linear':
                opt = f'-t 0 -d 2 -c {c} -q -v 5'
                optimal_cv_acc, optimal_opt = compareAccuracy(Y, X, opt, optimal_cv_acc, 
                    optimal_opt)
                cnt += 1
            elif k == 'polynomial':
                for gamma in gammas:
                    opt = f'-t 1 -d 2 -c {c} -q -v 5 -g {gamma}'
                    optimal_cv_acc, optimal_opt = compareAccuracy(Y, X, opt, optimal_cv_acc, 
                        optimal_opt)
                    cnt += 1
            elif k == 'RBF':
                for gamma in gammas:
                    opt = f'-t 2 -d 2 -c {c} -q -v 5 -g {gamma}'
                    optimal_cv_acc, optimal_opt = compareAccuracy(Y, X, opt, optimal_cv_acc, 
                        optimal_opt)
                    cnt += 1
    print(f'Total combinations: {cnt}')
    print(f'Optimal cross validation accuracy: {optimal_cv_acc}')
    print(f'Optimal option: {optimal_opt}')

In [8]:
gridSearch(X_train, Y_train)

-t 0 -d 2 -c 0.001 -q -v 5
Cross Validation Accuracy = 95.48%
-t 0 -d 2 -c 0.01 -q -v 5
Cross Validation Accuracy = 97.04%
-t 0 -d 2 -c 0.1 -q -v 5
Cross Validation Accuracy = 96.82%
-t 0 -d 2 -c 1 -q -v 5
Cross Validation Accuracy = 96.26%
-t 0 -d 2 -c 10 -q -v 5
Cross Validation Accuracy = 95.9%
-t 1 -d 2 -c 0.001 -q -v 5 -g 0.001
Cross Validation Accuracy = 45.4%
-t 1 -d 2 -c 0.001 -q -v 5 -g 0.01
Cross Validation Accuracy = 45.36%
-t 1 -d 2 -c 0.001 -q -v 5 -g 0.1
Cross Validation Accuracy = 94.7%
-t 1 -d 2 -c 0.001 -q -v 5 -g 1
Cross Validation Accuracy = 98.04%
-t 1 -d 2 -c 0.01 -q -v 5 -g 0.001
Cross Validation Accuracy = 45.64%
-t 1 -d 2 -c 0.01 -q -v 5 -g 0.01
Cross Validation Accuracy = 80.5%
-t 1 -d 2 -c 0.01 -q -v 5 -g 0.1
Cross Validation Accuracy = 97.6%
-t 1 -d 2 -c 0.01 -q -v 5 -g 1
Cross Validation Accuracy = 98.14%
-t 1 -d 2 -c 0.1 -q -v 5 -g 0.001
Cross Validation Accuracy = 45.78%
-t 1 -d 2 -c 0.1 -q -v 5 -g 0.01
Cross Validation Accuracy = 94.72%
-t 1 -d 2 -c 0.1 -

In [18]:
opt = f"-t 1 -d 2 -c 10 -q -g 1"
m =  svm_train(Y_train, X_train, opt)
res = svm_predict(Y_test, X_test, m)

Accuracy = 97.68% (2442/2500) (classification)


# Part3 User-defined kernel

In [11]:
def linearKernel(X1, X2):
    kernel = X1 @ X2.T
    return kernel
def RBFKernel(X1, X2, gamma):
    dist = np.sum(X1 ** 2, axis=1).reshape(-1, 1) + np.sum(X2 ** 2, axis=1) - 2 * X1 @ X2.T
    kernel = np.exp((-1 * gamma * dist))
    return kernel

In [25]:
linear_kernel = linearKernel(X_train, X_train)
RBF_kernel = RBFKernel(X_train, X_train, 1 / 784)
linear_kernel_s = linearKernel(X_train, X_test).T
RBF_kernel_s = RBFKernel(X_train, X_test, 1 / 784).T
# normalize linear_kernel to (0, 1)
linear_kernel = (linear_kernel - np.full((5000, 5000), linear_kernel.min()))/(np.full((5000, 5000), linear_kernel.max()) - np.full((5000, 5000), linear_kernel.min()))
linear_kernel_s = (linear_kernel_s - np.full((2500, 5000), linear_kernel_s.min()))/(np.full((2500, 5000), linear_kernel_s.max()) - np.full((2500, 5000), linear_kernel_s.min()))
# add index
X_kernel = np.hstack((np.arange(1, 5001).reshape((-1, 1)), linear_kernel + RBF_kernel))
X_kernel_s = np.hstack((np.arange(1, 2501).reshape((-1, 1)), linear_kernel_s + RBF_kernel_s))
print(X_kernel)

[[1.00000000e+00 1.46701189e+00 1.40008426e+00 ... 1.02159800e+00
  1.04502355e+00 1.08924174e+00]
 [2.00000000e+00 1.40008426e+00 1.54082252e+00 ... 1.01590892e+00
  1.01381870e+00 1.10040517e+00]
 [3.00000000e+00 1.22901999e+00 1.22016227e+00 ... 1.00851938e+00
  1.19453931e+00 1.28616882e+00]
 ...
 [4.99800000e+03 1.02159800e+00 1.01590892e+00 ... 1.50638894e+00
  1.00498480e+00 1.06890534e+00]
 [4.99900000e+03 1.04502355e+00 1.01381870e+00 ... 1.00498480e+00
  1.36803192e+00 1.16734227e+00]
 [5.00000000e+03 1.08924174e+00 1.10040517e+00 ... 1.06890534e+00
  1.16734227e+00 1.68636910e+00]]


In [27]:
opt = '-t 4 -q -d 2'
m = svm_train(Y_train, X_kernel, opt)
svm_predict(Y_test, X_kernel_s, m)

Accuracy = 95.88% (2397/2500) (classification)


([1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  3.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  5.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  5.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  3.0,
  1.0,
  1.0,
  1.0,